In [3]:
from astrosite_dataset import AstrositeDataset

dataset_path = 'data/astrosite/recordings'
target_id = 50574
dataset = AstrositeDataset(dataset_path, split=str(target_id))
len(dataset)

39

Collect 9 samples that have only 1 satellite label, and only retain the events that are labelled as that satellite.

In [4]:
orig_samples = []
sat_samples = []
n_samples = 9

for sample in dataset:
    labels = sorted(list(set(sample['labelled_events']['label'])))
    if labels[0] < -1:
        print(f"Skipping sample with labels {labels}")
        continue
    events = sample['events']
    orig_samples.append(events)
    sat_events = sample['labelled_events']
    sat_events = sat_events[sat_events['label'] == -1]
    sat_samples.append(sat_events)
    if len(sat_samples) >= n_samples:
        break

In [5]:
len(sat_samples)

9

In [6]:
import numpy as np

side_length = np.sqrt(n_samples).astype(int)
max_x = 1280
max_y = 720

for i in range(side_length):
    for j in range(side_length):
        index = i*side_length+j
        x_shift = max_x*j
        y_shift = max_y*i
        sat_samples[index]['x'] += x_shift
        sat_samples[index]['y'] += y_shift
        orig_samples[index]['x'] += x_shift
        orig_samples[index]['y'] += y_shift

merged_sat_events = np.concatenate(sat_samples)
merged_sat_events = np.sort(merged_sat_events, order='t')
merged_orig_events = np.concatenate(orig_samples)
merged_orig_events = np.sort(merged_orig_events, order='t')

In [7]:
import event_stream

event_stream.Encoder('merged_sat_events.es', 'dvs', 1280*side_length, 720*side_length).write(merged_sat_events)
event_stream.Encoder('merged_orig_events.es', 'dvs', 1280*side_length, 720*side_length).write(merged_orig_events)

In [23]:
target_sat_events = []

for sample in dataset:
    labels = sorted(list(set(sample['labelled_events']['label'])))
    if labels[0] < -1:
        print(f"Skipping sample with labels {labels}")
        continue
    
    events = sample['labelled_events']
    sat_events = events[events['label'] == -1]
    if len(sat_events) > 320:
        target_sat_events.append(sat_events)

Skipping sample with labels [-2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]
Skipping sample with labels [-2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]


In [24]:
len(target_sat_events)

9